In [197]:
import mne
import pandas as pd
from tqdm import tqdm

In [198]:
file_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_eeg.bdf'
eeg_data = mne.io.read_raw_bdf(file_path, preload=True)

channels_tsv_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_channels.tsv'
channels_info = pd.read_csv(channels_tsv_path, sep='\t')

events_tsv_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_events.tsv'
events_info = pd.read_csv(events_tsv_path, sep='\t')

Extracting EDF parameters from /Users/omaraguilarjr/GATech/Project-InnerSpeech/Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 696319  =      0.000 ...  1359.998 secs...


In [199]:
# Sort data into their channels
available_channels = set(eeg_data.ch_names).intersection(set(channels_info['name']))
eeg_data.pick_channels(list(available_channels))
montage = mne.channels.make_standard_montage('biosemi64')
eeg_data.set_montage(montage, on_missing='ignore')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawEDF | sub-01_ses-EEG_task-inner_eeg.bdf, 60 x 696320 (1360.0 s), ~318.8 MB, data loaded>

In [200]:
# Function used to bin data
def bin_eeg_data(eeg_data, events_info):
    events_data = events_info.copy()
    events_data['onset'] = events_info['onset'] / 1000
    events_data['duration'] = events_info['duration'] / 1000

    headers = ['label', 'label_type'] + eeg_data.ch_names
    binned_data = []

    for _, event in tqdm(events_data.iterrows(), total=len(events_data), desc='EEG Event Binning'):
        if event['trial_type'] in ['wife', 'child', 'father', 'daughter']:
            label_type = 'social'
        elif event['trial_type'] in ['four', 'three', 'ten', 'six']:
            label_type = 'numeric'
        else:
            label_type = 'unknown'

        onset = event['onset']
        duration = event['duration']
        label = event['trial_type']
        
        start_sample = int(onset * eeg_data.info['sfreq'])
        stop_sample = int((onset + duration) * eeg_data.info['sfreq'])

        event_dict = {'label': label, 'label_type': label_type}
        for channel in eeg_data.info['ch_names']:
            event_dict[channel] = eeg_data.get_data(picks=channel, start=start_sample, stop=stop_sample)[0].tolist()
        
        binned_data.append(event_dict)

    binned_df = pd.DataFrame(binned_data, columns=headers)
    return binned_df

In [202]:
binned_df = bin_eeg_data(eeg_data, events_info)

EEG Event Binning: 100%|██████████| 319/319 [00:31<00:00, 10.02it/s]


In [203]:
binned_df.head

<bound method NDFrame.head of       label label_type                                                Fpz  \
0       six    numeric  [0.005213037241699531, 0.0052121934932585655, ...   
1       six    numeric  [0.005296818336893221, 0.00529728708602709, 0....   
2      four    numeric  [0.005464443027165116, 0.005461286782997059, 0...   
3       ten    numeric  [0.005338037010731519, 0.005339630757786676, 0...   
4      four    numeric  [0.005266662142614254, 0.005265693394404256, 0...   
..      ...        ...                                                ...   
314    four    numeric  [0.005125474903492623, 0.005125631153203913, 0...   
315    four    numeric  [0.0050674125107772645, 0.00506422501666695, 0...   
316   three    numeric  [0.0051289748970255195, 0.005134912386054538, ...   
317  father     social  [0.00509853745326623, 0.005101912447030094, 0....   
318  father     social  [0.005135256135419377, 0.005135287385361635, 0...   

                                             

In [204]:
binned_df.to_csv('../../Data/Processed/sub01_binned.csv', index=False)